In [1]:
# importing packages
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from numpy.linalg import cond, matrix_rank, norm
from scipy.linalg import inv, solve, det, eig, lu, eigvals
from sklearn.metrics import confusion_matrix

In [119]:
#importing data
BL = pd.read_csv('Data/BankLend.csv')
consumption = pd.read_csv('Data/consumption.csv')
gdp_exp = pd.read_csv('Data/gdp.csv')
housing_starts = pd.read_csv('Data/HousingStarts.csv')
ex_rate = pd.read_csv('Data/CEER_MONTHLY_NOMINAL-sd-1990-01-01.csv') #Canadian Effective Exchange Rate - monthly nominal
SP_tbills = pd.read_csv('Data/s&p, tbills.csv')
house_prices = pd.read_csv('Data/OECD Data.csv')

In [130]:
# BANK LENDING

# Summing over rows of 3 to get quarterly data
def sum_quarter(group):
    return pd.Series({'ReferencePeriod': group['ReferencePeriod'].iloc[0],
                      'Total, funds advanced, residential mortgages, insured 7': group['Total, funds advanced, residential mortgages, insured 7'].sum(),
                      'Total, funds advanced, residential mortgages, uninsured 7': group['Total, funds advanced, residential mortgages, uninsured 7'].sum(),
                      'Total, funds advanced, variable rate mortgages': group['Total, funds advanced, variable rate mortgages'].sum(),
                      'Total, funds advanced for non-mortgage loans, consumer credit': group['Total, funds advanced for non-mortgage loans, consumer credit'].sum(),
                      'Total, funds advanced, business loans 8': group['Total, funds advanced, business loans 8'].sum()})

# Group by sets of 3 rows and average using the custom function
BankLend = BL.groupby(BL.index // 3).apply(sum_quarter)
BankLend['ReferencePeriod'] = pd.to_datetime(BankLend['ReferencePeriod'])
BankLend['Quarters'] = BankLend['ReferencePeriod'].dt.to_period("Q").astype(str)

# cleaning up the data
BankLend = BankLend.drop(['ReferencePeriod'], axis=1)
BankLend.head()

,"Total, funds advanced, residential mortgages, insured 7","Total, funds advanced, residential mortgages, uninsured 7","Total, funds advanced, variable rate mortgages","Total, funds advanced for non-mortgage loans, consumer credit","Total, funds advanced, business loans 8",Quarters
0,19334,31399,0.0,40205,145993,2013Q1
1,31104,49731,0.0,50881,181034,2013Q2
2,27995,49720,0.0,50063,177620,2013Q3
3,20997,37640,0.0,44350,200198,2013Q4
4,19096,32194,0.0,42025,163412,2014Q1


In [121]:
# GDP 

# keeping specific columns
gdp = gdp_exp[['Quarters', 'Gross domestic product at market prices']]

gdp.head()

,Quarters,Gross domestic product at market prices
0,2000Q1,"1,514,676"
1,2000Q2,"1,532,503"
2,2000Q3,"1,548,212"
3,2000Q4,"1,551,274"
4,2001Q1,"1,560,006"


In [122]:
# EXCHANGE RATE

# keeping one column
filtered_ex_rate = ex_rate[['date','CEER_BROADNM']]
filtered_ceer = filtered_ex_rate.reset_index(drop=True)

# Averaging over rows of 3 to get quarterly data
def average_quarter(group):
    return pd.Series({'date': group['date'].iloc[0],
                      'CEER_BROADNM': group['CEER_BROADNM'].mean()})

# Group by sets of 3 rows and average using the custom function
ceer = filtered_ceer.groupby(filtered_ceer.index // 3).apply(average_quarter).reset_index(drop=True)

# converting date to datetime format
ceer['date'] = pd.to_datetime(ceer['date'])
ceer['Quarters'] = ceer['date'].dt.to_period("Q").astype(str)

ceer = ceer.drop(['date'], axis=1)
ceer.head()

,CEER_BROADNM,Quarters
0,98.053333,1999Q1
1,101.366667,1999Q2
2,100.096667,1999Q3
3,100.486667,1999Q4
4,102.433333,2000Q1


In [134]:
# CONSUMPTION, HOUSE PRICES, HOUSING STARTS

print(consumption.head())
print(house_prices.head())
print(housing_starts.head())

  Quarters  Household final consumption expenditure
0   2000Q1                                   143884
1   2000Q2                                   145903
2   2000Q3                                   148957
3   2000Q4                                   150284
4   2001Q1                                   151594
  Quarters  House_Prices
0   2001Q1     40.844646
1   2001Q2     41.200980
2   2001Q3     41.875943
3   2001Q4     42.620664
4   2002Q1     43.340875
  Quarters HousingStarts
0   2016Q1        38,175
1   2016Q2        53,286
2   2016Q3        54,899
3   2016Q4        51,555
4   2017Q1        42,862


In [137]:
# FINANCIAL INDICATORS - S&P 500 & Treasury Bills

# Averaging over rows of 3 to get quarterly data
def average_quarter(group):
    return pd.Series({'nummonth': group['nummonth'].iloc[0],
                      'sp': group['sp'].mean(), 't_bills': group['t_bills'].mean()})

# Group by sets of 3 rows and average using the custom function
fin_ind_quarter = SP_tbills.groupby(SP_tbills.index // 3).apply(average_quarter).reset_index(drop=True)

# creating the time series column
data = {'date': pd.date_range(start='2013-01-01', end='2022-10-01', freq='M')}
df = pd.DataFrame(data)

# Turning it into a quarterly periods
df['quarterly_period'] = df['date'].dt.to_period('Q')
df['formatted_quarter'] = df['quarterly_period'].dt.strftime('%YQ%q')

# concatinating the dataframe with the financial indicators dataframe
fin_ind = pd.concat([df, fin_ind_quarter], axis=1)

# cleaning up the dataframe
drop_columns = ['date', 'quarterly_period', 'nummonth']
fin_ind = fin_ind.drop(drop_columns, axis=1)
fin_ind = fin_ind.rename(columns = {"formatted_quarter":"Quarters"})
fin_ind.head()

,Quarters,sp,t_bills
0,2013Q1,36782.440000,0.953333
1,2013Q1,36089.766667,1.010000
2,2013Q1,37059.500000,0.990000
3,2013Q2,39750.513333,0.916667
4,2013Q2,41900.943333,0.876667


In [152]:
merge1 = pd.merge(gdp, Consumption, on = 'Quarters')
merge2 = pd.merge(merge1, BankLend, on = 'Quarters')
merge3 = pd.merge(merge2, ceer, on = 'Quarters')
merge4 = pd.merge(merge3, house_prices, on = 'Quarters')
merge5 = pd.merge(merge4, housing_starts, on = 'Quarters')
merge6 = pd.merge(merge5, fin_ind, on = 'Quarters')
merge6

,Quarters,Gross domestic product at market prices,Household final consumption expenditure,"Total, funds advanced, residential mortgages, insured 7","Total, funds advanced, residential mortgages, uninsured 7","Total, funds advanced, variable rate mortgages","Total, funds advanced for non-mortgage loans, consumer credit","Total, funds advanced, business loans 8",CEER_BROADNM,House_Prices,HousingStarts,sp,t_bills
0,2016Q1,"2,071,732",285296,27018,51730,0.0,46899,185088,111.046667,105.128205,"38,175",80737.096667,0.423333
1,2016Q1,"2,071,732",285296,27018,51730,0.0,46899,185088,111.046667,105.128205,"38,175",76542.553333,1.593333
2,2016Q1,"2,071,732",285296,27018,51730,0.0,46899,185088,111.046667,105.128205,"38,175",73474.410000,3.140000
3,2016Q2,"2,061,412",286760,36591,69583,0.0,58571,207527,117.166667,108.601056,"53,286",76397.386667,4.056667
4,2016Q2,"2,061,412",286760,36591,69583,0.0,58571,207527,117.166667,108.601056,"53,286",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2022Q2,"2,327,881",369243,23204,116776,73180.0,79789,332027,123.320000,182.458522,"72,314",NaN,NaN
77,2022Q2,"2,327,881",369243,23204,116776,73180.0,79789,332027,123.320000,182.458522,"72,314",NaN,NaN
78,2022Q3,"2,338,543",372452,17160,88049,47401.0,68348,326879,123.330000,177.132353,"73,548",NaN,NaN
79,2022Q3,"2,338,543",372452,17160,88049,47401.0,68348,326879,123.330000,177.132353,"73,548",NaN,NaN
